In [54]:
from instagram_private_api import Client, ClientCompatPatch
import api_requests
import importlib
importlib.reload(api_requests)
from api_requests import (BasicUserInfo, PublicUserInfoRequests, MediaInfoRequests, 
                            InteractWithUsersActions, CollectDataThroughHashtags)

import time

In [2]:
"""
Only run once, multiple log ins will cause account ban
TODO: Figure out how to cache auth cookie details. Hints: Can call api.cookie_jar, look into client.py
"""
username = 'cuteanimalzzzz123'
password = 'instabotOP'
api = Client(username, password)

In [8]:
test_usernames = ['ryanlopezzzz', #small size public account example
                  'ruining99', #small size public account example
                  'i_am_jorge_murillo', #private account example
                  'cuteanimalzzzz123' #API login account
                 ]
large_test_usernames = ['cute.animals'] #large size public account

# Test BasicUserInfo() Class

In [10]:
def test_basic_user_info(username):
    """
    Can check against true values by going onto Instagram app.
    """
    basic_user_info = BasicUserInfo(api, username)
    print("User Id: " + str(basic_user_info.user_id))
    print("Follow count: " + str(basic_user_info.follower_count))
    print("Following count: " + str(basic_user_info.following_count))
    print("Media count: " + str(basic_user_info.media_count))
    print("Private status: " + str(basic_user_info.private_status))

In [11]:
for username in test_usernames + large_test_usernames:
    print("Username: " + username)
    test_basic_user_info(username)
    print("-"*100)

Username: ryanlopezzzz
User Id: 255272463
Follow count: 635
Following count: 390
Media count: 14
Private status: False
----------------------------------------------------------------------------------------------------
Username: ruining99
User Id: 2142436680
Follow count: 387
Following count: 368
Media count: 15
Private status: False
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo
User Id: 9984230653
Follow count: 173
Following count: 191
Media count: 2
Private status: True
----------------------------------------------------------------------------------------------------
Username: cuteanimalzzzz123
User Id: 50004795460
Follow count: 0
Following count: 0
Media count: 0
Private status: False
----------------------------------------------------------------------------------------------------
Username: cute.animals
User Id: 435102802
Follow count: 88282
Following count: 1
Media count: 344
Private status: F

# Test PublicUserInfoRequests() Class

In [6]:
def test_public_user_info_requests(username):
    puir = PublicUserInfoRequests(api,username,rank_token)
    
    followers_usernames = puir.get_followers_usernames()
    assert(puir.follower_count == len(followers_usernames))
    print('Loaded %d follower usernames.'%len(followers_usernames))
    print('Peeking at first 3 followers: '+str(followers_usernames[:3]))
    print()    
    following_usernames = puir.get_following_usernames()
    assert(puir.following_count == len(following_usernames))
    print('Loaded %d following usernames.'%len(following_usernames))
    print('Peeking at first 3 following: '+str(following_usernames[:3]))
    print()
    user_feed_media_ids = puir.get_user_feed_media_ids()
    assert(puir.media_count == len(user_feed_media_ids))
    print('Loaded %d media (posts) ids.'%len(user_feed_media_ids))
    print('Peeking at first 2 ids: ' + str(user_feed_media_ids[:2]))

In [7]:
for username in test_usernames:
    print("Username: " + username + '\n')
    try:
        test_public_user_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Username: ryanlopezzzz

Loaded 636 follower usernames.
Peeking at first 3 followers: ['c.hirish', 'matthewkim812', 'r.yamas']

Loaded 391 following usernames.
Peeking at first 3 following: ['c.hirish', 'happy_lucky_now.jp', 'neha_upponi']

Loaded 14 media (posts) ids.
Peeking at first 2 ids: ['2112726015003571292_255272463', '2018080158682059371_255272463']
----------------------------------------------------------------------------------------------------
Username: ruining99

Loaded 387 follower usernames.
Peeking at first 3 followers: ['cassandra.chow', 'c.hirish', 'andrewarmen']

Loaded 368 following usernames.
Peeking at first 3 following: ['cassandra.chow', 'c.hirish', 'johannalimmer']

Loaded 15 media (posts) ids.
Peeking at first 2 ids: ['2601326525749798387_2142436680', '2580166803319609035_2142436680']
----------------------------------------------------------------------------------------------------
Username: i_am_jorge_murillo

Exception: ValueError, Message: Can't get foll

# Test MediaInfoRequests() Class
Seems there can be an error where it loads less comments than get_comment_count() says there are.

In [27]:
def test_media_info_requests(username):
    #Get user's most recent media id
    puir = PublicUserInfoRequests(api,username)
    user_feed_media_ids = puir.get_user_feed_media_ids()
    most_recent_media_id = user_feed_media_ids[0]
    
    mir = MediaInfoRequests(api,most_recent_media_id)
    print('Comment count: %d'%mir.get_comment_count())
    print()                       
    comments_text = mir.get_n_comments_text()
    print('Loaded %d comments text.'%len(comments_text))
    print('Peeking at first 2 comments: '+str(comments_text[:2]))
    print()
    comment_counter = mir.get_n_comment_count_per_username()
    assert(sum(comment_counter.values())==len(comments_text)) 
    print('Loaded %d (username, comment count) data.'%len(comment_counter))
    comment_counter_list = list(comment_counter.items())
    print('Peeking at first 2 elements: '+str(comment_counter_list[:2]))

In [28]:
for username in test_usernames: #+ large_test_usernames:
    print("Testing on %s's first post: \n"%username)
    try:
        test_media_info_requests(username)
    except Exception as e:
        print('Exception: %s, Message: %s'%(type(e).__name__,e))
    print("-"*100)

Testing on ryanlopezzzz's first post: 

Comment count: 7

Exception: Warning, Message: _get_comments_info may not load all comments
----------------------------------------------------------------------------------------------------
Testing on ruining99's first post: 

Comment count: 2

Exception: Warning, Message: _get_comments_info may not load all comments
----------------------------------------------------------------------------------------------------
Testing on i_am_jorge_murillo's first post: 

Exception: ValueError, Message: Can't get following info from private account.
----------------------------------------------------------------------------------------------------
Testing on cuteanimalzzzz123's first post: 

Exception: ValueError, Message: Strange behavior when requesting your own info, needs further testing.
----------------------------------------------------------------------------------------------------


# Test InteractWithUsersActions() Class

In [49]:
def test_interact_with_users_actions(username):
    """
    Can check by looking onto the Instagram app when function is called
    """
    #Get user's id and oldest media id
    puir = PublicUserInfoRequests(api,username)
    user_id = puir.user_id
    user_feed_media_ids = puir.get_user_feed_media_ids()
    oldest_media_id = user_feed_media_ids[-1]
    
    iwua = InteractWithUsersActions(api)
    print('Following user %s...'%username)
    iwua.follow_user(user_id)
    print('Successfully followed - now sleeping')
    time.sleep(30) #can check that this user was followed in this time
    print()
    print('Unfollowing user')
    iwua.unfollow_user(user_id)
    print('Successfully unfollowed - now sleeping')
    time.sleep(10)
    print()
    print('Liking %s last media post'%username)
    iwua.like_post(oldest_media_id)
    print('Successfully liked - now sleeping')
    time.sleep(10)
    print()
    comment = 'Great photo!'
    print('Posting comment \'%s\' on %s last post'%(comment,username))
    iwua.post_comment(oldest_media_id,comment)
    print('Successfully posted comment.')

In [50]:
test_actions_username = test_usernames[0]
print('Testing on user %s: \n'%test_actions_username)
test_interact_with_users_actions(test_actions_username)

Testing on user ryanlopezzzz: 

Following user ryanlopezzzz...
Successfully followed - now sleeping

Unfollowing user
Successfully unfollowed - now sleeping

Liking ryanlopezzzz last media post
Successfully liked - now sleeping

Posting comment 'Great photo!' on ryanlopezzzz last post
Successfully posted comment.


# Below is Uncompleted Code

In [55]:
rank_token = api.generate_uuid()

In [70]:
loaded_feed = api.feed_tag('banngood', rank_token)

In [75]:
for key, value in loaded_feed.items():
    print(key)
    print(value)
    print('-'*80)

items
[{'taken_at': 1631387230, 'pk': 2660591385323521540, 'id': '2660591385323521540_3946108185', 'device_timestamp': 1631387190172, 'media_type': 1, 'code': 'CTsUcS-rEYE', 'client_cache_key': 'MjY2MDU5MTM4NTMyMzUyMTU0MA==.2', 'filter_type': 0, 'is_unified_video': False, 'user': {'pk': 3946108185, 'username': 'gadgetsfromchina', 'full_name': 'GadgetsFromChina', 'is_private': False, 'profile_pic_url': 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-19/s150x150/23507476_1586715284724742_3052092945533501440_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_ohc=tqQHH-BZ0QIAX_BAujX&edm=AMT5gP0BAAAA&ccb=7-4&oh=1a9878ffeb327008a40fbc36761e722d&oe=617BA820&_nc_sid=d50af0', 'profile_pic_id': '1341223683821444867_3946108185', 'friendship_status': {'following': False, 'outgoing_request': False, 'is_bestie': False, 'is_restricted': False, 'is_feed_favorite': False}, 'has_anonymous_profile_picture': False, 'is_unpublished': False, 'is_favorite': False, 'account_badges': []}, 'can_viewer_resha

In [78]:
print(len(loaded_feed['items']))
print(loaded_feed['items'][0])

9
{'taken_at': 1631387230, 'pk': 2660591385323521540, 'id': '2660591385323521540_3946108185', 'device_timestamp': 1631387190172, 'media_type': 1, 'code': 'CTsUcS-rEYE', 'client_cache_key': 'MjY2MDU5MTM4NTMyMzUyMTU0MA==.2', 'filter_type': 0, 'is_unified_video': False, 'user': {'pk': 3946108185, 'username': 'gadgetsfromchina', 'full_name': 'GadgetsFromChina', 'is_private': False, 'profile_pic_url': 'https://scontent-lax3-1.cdninstagram.com/v/t51.2885-19/s150x150/23507476_1586715284724742_3052092945533501440_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_ohc=tqQHH-BZ0QIAX_BAujX&edm=AMT5gP0BAAAA&ccb=7-4&oh=1a9878ffeb327008a40fbc36761e722d&oe=617BA820&_nc_sid=d50af0', 'profile_pic_id': '1341223683821444867_3946108185', 'friendship_status': {'following': False, 'outgoing_request': False, 'is_bestie': False, 'is_restricted': False, 'is_feed_favorite': False}, 'has_anonymous_profile_picture': False, 'is_unpublished': False, 'is_favorite': False, 'account_badges': []}, 'can_viewer_reshare': 

In [79]:
for key, value in loaded_feed['items'][0].items():
    print(key)
    print(value)
    print('-'*80)

taken_at
1631387230
--------------------------------------------------------------------------------
pk
2660591385323521540
--------------------------------------------------------------------------------
id
2660591385323521540_3946108185
--------------------------------------------------------------------------------
device_timestamp
1631387190172
--------------------------------------------------------------------------------
media_type
1
--------------------------------------------------------------------------------
code
CTsUcS-rEYE
--------------------------------------------------------------------------------
client_cache_key
MjY2MDU5MTM4NTMyMzUyMTU0MA==.2
--------------------------------------------------------------------------------
filter_type
0
--------------------------------------------------------------------------------
is_unified_video
False
--------------------------------------------------------------------------------
user
{'pk': 3946108185, 'username': 'gadgetsfrom

In [82]:
post_number = 2
print('Username:')
print(loaded_feed['items'][post_number]['user']['username'])

Username:
perthwestoz_rc
